<a href="https://colab.research.google.com/github/Ohmyprogram/Delivery-Robot-TextMining/blob/main/%EA%B0%90%EC%84%B1%EC%9D%BC%EA%B8%B0_LDA_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 코랩 환경 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 디렉토리 변경
%cd drive/MyDrive/혁신기술텍스트마이닝

/content/drive/MyDrive/혁신기술텍스트마이닝


In [ ]:
# 현재 디렉토리 경로 확인
!pwd

/content/drive/MyDrive/혁신기술텍스트마이닝


# 라이브러리

In [ ]:
# !pip install PyKomoran

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip install git+https://github.com/ssut/py-hanspell.git
# !pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-boq6m10t
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-boq6m10t
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=7410839c411853a2879b24b71bf920807b66025d21ba3868eaa59209e256956d
  Stored in directory: /tmp/pip-ephem-wheel-cache-pdqy32p3/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import io
import pandas as pd
from wordcloud import WordCloud
from collections import Counter
from PyKomoran import *
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import regex
import platform
import matplotlib.font_manager as fm
import re
from hanspell import spell_checker
from tqdm import tqdm

# 데이터 전처리

In [ ]:
raw_data = pd.read_excel('/content/drive/MyDrive/혁신기술텍스트마이닝/data/raw_data.xlsx')
raw_data.head()

,느낀점
0,시도한지 20분이 지나도 배차가 되지 않아서 결국 버스로 환승
1,호출했으나 이번에도 배차가 30분 정도 되지 않음
2,운좋게 배차가 됐고 약 20분 후에 도착한다고 바로 알림
3,예상 시간과 비슷한 시간에 탑승
4,QR코드를 기기에 태그하기만 하면 돼서 간편


In [ ]:
# nan값 제거,맞춤법,'";"등 . , 제외한 다른 기호 제거
raw_data.isnull().sum()

느낀점    7
dtype: int64

In [ ]:
clean_df = raw_data.dropna()
clean_df.isnull().sum()

느낀점    0
dtype: int64

In [ ]:
# 텍스트 전처리
def cleanText(text):
  specialChar = '[-=+#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》❤♥♡ㅏㅑㅓㅑㅗㅛㅜㅠㅢㅡㅣㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㄲㄸㅆㅉㅃ]'
  text = ''.join(c for c in text if c not in specialChar)
  only_BMP_pattern = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외 (이모지 제거)
                           "]+", flags=re.UNICODE)
  return only_BMP_pattern.sub(r'', text)

# 맞춤법 검사
def spell_check(comments):
  for k in tqdm(range(len(comments))):
    comments[k] = cleanText(comments[k])
    spelled_sent = spell_checker.check(comments[k])
    checked_sent = spelled_sent.checked
    comments[k] = checked_sent
  print("맞춤법 검사 완료")

  #return comments.to_csv(bank_name+'_spell_checked.csv')


spell_check(clean_df['comment'])
clean_df.to_csv("comments_spell_checked.csv")

In [ ]:
# documents=[
#     '점심을 주문하기 위해 요기요에 들어가서 메뉴를 보고 짜장면과 탕수육을 대연각에서 주문하였다. 대연각은 로봇과 연계 배송을 하는지 기존 주문 프로세스와 다르게 라이더 배정 및 로봇 배차 정보가 전송되었다. 로봇이 어떻게 배송을 해주는 것인지에 대해 자세한 설명이 없어 주문이 잘 된 것인지, 잘못 된 것인지 모호했고, 기존 프로세스와 조금 상이하여 음식이 잘못 올 수도 있겠다는 우려가 조금 있었다. 우선 처음 경험하는 로봇 배달이었기에 생소하여 이에 대한 자세한 설명과 프로세스에 대한 정보를 팝업 혹은 안내 형태로 알려주었으면 좋겠다는 생각이 들었다.',
#     '라이더가 물품을 픽업 후 배달을 시작했다는 알림을 받았고, 기존 주문 배달 프로세스와 동일하게 라이더의 배달 이동경로 확인이 가능하여 이 단계에서는 크게 불편한 부분을 느끼지 못하였다. 하지만 로봇 배송이 시작되었는지 로봇의 배송 도착 안내에 대한 문자를 받았을 때, 조금 불편함을 느꼈다. 라이더가 물품을 픽업하는 부분에 대해서는 카카오톡으로 알림이 오고 이것이 요기요 앱과 연동되어 버튼 하나로 바로 앱으로 연동되지만 로봇 배달에 대한 안내는 문자로 수신받아 카카오톡, 요기요, 문자 창까지 삼중으로 켜놓아야 했다. 로봇 배달에 대한 안내 또한 요기요 앱 혹은 카카오톡으로 수신받으면 좋을 듯 하다. 또한 로봇의 도착 소요시간과 이동경로 확인이 어려워 라이더가 배달하는 프로세스보다 훨씬 불편했다.',
#     '로봇이 도착했다는 전화가 왔었는데, 당시에 내가 화장실에 가있어 전화를 받지 못하였다. 로봇의 대략적인 도착시간 또한 안내 받지 못하였기 때문에 언제쯤 도착할 것이라는 시간대를 전혀 몰라 로봇의 도착 전화를 15분 후에 확인할 수 있었다. 물품을 수령하러 로봇 도착 포인트로 가 문자로 수신받았던 본인인증 번호를 입력하였고 로봇 물품칸이 열려 음식을 수령하였다.',
#     '전화를 못받았던 터라 음식이 다 식었고, 불어있었다. 로봇이 도착했다고 안내 전화가 왔었는데 내가 전화를 받지 못하여 음식 상태가 안좋은 것 같아 반품 요청을 하기가 애매하여 우선 수령완료를 로봇 터치패드에서 눌렀다. 주문 물품 및 음식에 대한 반품 기준이 필요할 듯 하였다. 과실이 누구인지가 애매하여 이러한 부분은 명확한 기준이 필요할 것으로 보인다. 또한, 로봇 배송의 이점은 배달 소요시간인 듯 한데, 소요시간이 오히려 오래걸린 듯 하고, 서비스 프로세스 자체가 너무 생소하여 이에 대한 정확한 안내가 필요해보인다. 로봇의 경로 표시 또한 해주어야 할 듯 하다.'
# ]

# sentences = []                                                                                #단문들 담을 리스트
# for doc in documents:
#   sen = doc.split('.')
#   sen.pop()
#   sentences = sentences+sen

# df = pd.DataFrame({'Content' : sentences})

# df

,Content
0,점심을 주문하기 위해 요기요에 들어가서 메뉴를 보고 짜장면과 탕수육을 대연각에서 주...
1,대연각은 로봇과 연계 배송을 하는지 기존 주문 프로세스와 다르게 라이더 배정 및 ...
2,로봇이 어떻게 배송을 해주는 것인지에 대해 자세한 설명이 없어 주문이 잘 된 것인...
3,우선 처음 경험하는 로봇 배달이었기에 생소하여 이에 대한 자세한 설명과 프로세스에...
4,"라이더가 물품을 픽업 후 배달을 시작했다는 알림을 받았고, 기존 주문 배달 프로세스..."
5,"하지만 로봇 배송이 시작되었는지 로봇의 배송 도착 안내에 대한 문자를 받았을 때,..."
6,라이더가 물품을 픽업하는 부분에 대해서는 카카오톡으로 알림이 오고 이것이 요기요 ...
7,로봇 배달에 대한 안내 또한 요기요 앱 혹은 카카오톡으로 수신받으면 좋을 듯 하다
8,또한 로봇의 도착 소요시간과 이동경로 확인이 어려워 라이더가 배달하는 프로세스보다...
9,"로봇이 도착했다는 전화가 왔었는데, 당시에 내가 화장실에 가있어 전화를 받지 못하였다"


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/혁신기술텍스트마이닝/data/comments_spell_checked.csv")
df = df.drop(['Unnamed: 0'], axis = "columns")
df.rename(columns = {"느낀점":"Content"}, inplace = True)
df.head()

,Content
0,시도한지 20분이 지나도 배차가 되지 않아서 결국 버스로 환승
1,호출했으나 이번에도 배차가 30분 정도 되지 않음
2,운 좋게 배차가 됐고 약 20분 후에 도착한다고 바로 알림
3,예상 시간과 비슷한 시간에 탑승
4,QR코드를 기기에 태그 하기만 하면 돼서 간편


In [ ]:
# !pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import konlpy
import re

# tokenization 함수를 만들어둠
def tokenize_korean_text(text):
    text = re.sub(r'[^,.?!\w\s]','', text)  ## ,.?!와 문자+숫자+_(\w)와 공백(\s)만 남김  # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)

    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return

    words = []
    for word, pos in Okt_morphs:
        if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':  # 이 경우에는 형용사, 동사, 명사만 남김
            words.append(word)

    words_str = ' '.join(words)
    return words_str


# df['Content']를 하나씩 tokenize해서 list로 저장
tokenized_list = []

for text in df['Content']:
    tokenized_list.append(tokenize_korean_text(text))

print(len(tokenized_list))
print(tokenized_list[0])

366
시도 한지 지나도 배차 되지 않아서 버스 환승


In [ ]:
drop_corpus = []

for index in range(len(tokenized_list)):
    corpus = tokenized_list[index]
    if len(set(corpus.split())) < 3:   # 같은 단어 1-2개만 반복되는 corpus도 지우기 위해 set()을 사용
        df.drop(index, axis='index', inplace=True)
        drop_corpus.append(corpus)

for corpus in drop_corpus:
    tokenized_list.remove(corpus)

df.reset_index(drop=True, inplace=True)

print(len(tokenized_list))
print(len(df))

360
360


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#LDA 는 Count기반의 Vectorizer만 적용
count_vectorizer = CountVectorizer(ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
    # bigram도 포함

feat_vect = count_vectorizer.fit_transform(tokenized_list)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (360, 3236)


In [ ]:
lda = LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0, n_components=4)  # 토픽 수는 4개로 설정
lda.fit(feat_vect)

LatentDirichletAllocation(n_components=4)

In [ ]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환.
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]

        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
display_topics(lda, feature_names, 10)

Topic # 0
시간 버스 기사 이용 도착 예상 택시 예상 시간 호출 아쉬움
Topic # 1
버스 탑승 도착 목적지 시간 이동 필요 차량 안내 아이
Topic # 2
이용 배차 버스 호출 되지 느낌 택시 시간 정류장 도착
Topic # 3
시간 하는 이동 이용 서비스 정류장 버스 없어 목적지 경로


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# !pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyLDAvis.sklearn  # sklearn의 ldamodel에 최적화된 라이브러리

pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, feat_vect, count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
# # html로 저장
# pyLDAvis.save_html(vis, '감성일기예시_lda.html')

In [ ]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌

doc_topic = lda.transform(feat_vect)

doc_per_topic_list = []
for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    topic_pr = doc_topic[n].max()
    doc_per_topic_list.append([n, topic_most_pr, topic_pr])

doc_topic_df = pd.DataFrame(doc_per_topic_list, columns=['Doc_Num', 'Topic', 'Percentage'])

doc_topic_df.head()

,Doc_Num,Topic,Percentage
0,0,2,0.952581
1,1,2,0.945735
2,2,2,0.945166
3,3,1,0.579524
4,4,3,0.945146


In [ ]:
doc_topic_df = doc_topic_df.join(df)
doc_topic_df

,Doc_Num,Topic,Percentage,Content
0,0,2,0.952581,시도한지 20분이 지나도 배차가 되지 않아서 결국 버스로 환승
1,1,2,0.945735,호출했으나 이번에도 배차가 30분 정도 되지 않음
2,2,2,0.945166,운 좋게 배차가 됐고 약 20분 후에 도착한다고 바로 알림
3,3,1,0.579524,예상 시간과 비슷한 시간에 탑승
4,4,3,0.945146,QR코드를 기기에 태그 하기만 하면 돼서 간편
...,...,...,...,...
355,355,1,0.906035,멀미가 난다 승차감이 별로
356,356,1,0.945264,호출 대시 기간이 15분 일반 버스를 탈까 고민
357,357,1,0.924390,아쉬운 승차감에 계속해서 멀미
358,358,3,0.923420,멀미 때문에 차라리 택시를 탈 걸 후회


In [ ]:
# ⁣2. 토픽별 문서 수 계산
doc_topic_df.groupby('Topic')[['Doc_Num']].count()

,Doc_Num
Topic,
0,90
1,93
2,107
3,70


In [ ]:
len(doc_topic_df['Topic'].unique())

4

In [ ]:
# 토픽별로, 가장 높은 확률로 할당된 문서 top 3 확인
for topic in range(len(doc_topic_df['Topic'].unique())):
    print('Topic #', topic, '-----------------------------')
    top_pr_topics = doc_topic_df[doc_topic_df['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    n = 0
    for i in range(len(top_pr_topics['Content'])):
      print(top_pr_topics['Content'].iloc[i])
      n += 1
      if n == 3 :
        break;
    print('\n')

Topic # 0 -----------------------------
노선의 방법이나 위치 등 특이점을 찾아 합리적인 노선과 운용 방법이 있을 텐데 전혀 고려하지 않은 운영의 방법
목적지까지 정해진 시간에 무조건 도착해야 하는 상황이라면 IMOD는 좋지 않은 선택일 수도 있겠다는 생각이 들었음
입주한 가구가 많지 않아 교통이 원활하게 활성화되어있지 않은데 IMOD를 통해 가까운 거리를 편안하게 이동 가능해서 좋음


Topic # 1 -----------------------------
친목도모를 위한 아이모드 운영 산행 브런치 카페 모임 키즈카페 모임 운동모임 문화시설 모임 도 가능할 것 같다
운행이 불가능한 지역이면 모든 차량이 운행 중입니다’라는 문구 대신 운행이 불가능한 지역입니다’라는 느낌으로 문구를 바꾸는 것이 좋을듯함
완정 역에 도착하여 가방이 2개 있어 버스 타기도 애매하고 택시요금도 너무 부담되어 검단에서 운영 중인 IMOD를 호출하여 사용


Topic # 2 -----------------------------
배차 잡기 힘든 이용수단이라는 이미지가 각인되어 있는데 이용 구역을 벗어난 곳 선택해도 운행 중이라고 나오면 그 이미지가 더욱 단단히 박힐 것임
기존 교통수단 트래픽 이용해 검단 계양 or 계양 검단으로의 이동이 어느 정도 되는지 파악해 계양과 검단같이 붙어있는 도시들은 IMOD로 이용이 가능했으면 함 
버스 대기시간 이동시간 등을 고려하면 걷는 것이 더 빠를 수도 있고 비용적인 측면에서도 이득입니다만 홍보를 위해 이용


Topic # 3 -----------------------------
목상동 솔밭 정류소는 버스 시내버스 배차시간이 20분인데 도착시간 알려주는 LED 표지판도 없어 짜증을 내며 무작정 기다리는 시민도 있었음
코로나 시대에 다른 사람과 합승하는 것이 좀 걸리지만 내부에 손소독기 체온계가 있어 코로나로부터 보호받는 것 같았음
 기존 상권이 형성되어 있는 원도심 구도심으로 이동하였음 해당 노선을 이동하는 부분에선 